# 2024-05-19

## 1378. Replace Employee ID With The Unique Identifier

In [ ]:
import pandas as pd

def replace_employee_id(employees: pd.DataFrame, employee_uni: pd.DataFrame) -> pd.DataFrame:
    result = pd.merge(employee_uni, employees, how='right', on='id').drop('id', axis=1)
    return result

# 2024-05-20

## 1407. Top Travellers

In [ ]:
import pandas as pd

def top_travellers(users: pd.DataFrame, rides: pd.DataFrame) -> pd.DataFrame:
    # user별 총 주행 거리 계산
    total_distance  = rides.groupby('user_id')['distance'].sum().reset_index()
    
    # merge하여 user 정보와 총 주행 거리를 포함하는 df 생성
    merged = pd.merge(users, total_distance, left_on='id', right_on='user_id', how='left')[['name', 'distance']]
    
    # NaN을 0으로 대체하고 문제에서 제시한 기준대로 정렬
    result = merged.fillna(0).sort_values(by=['distance', 'name'], ascending=[False, True])
    
    result.rename(columns={'distance': 'travelled_distance'}, inplace=True)
    return result

# 2024-05-21

## 1484. Group Sold Products By The Date

In [ ]:
import pandas as pd

def categorize_products(activities: pd.DataFrame) -> pd.DataFrame:
    result = activities.groupby('sell_date')['product'].agg(
        # 고유 제품 수 계산
        num_sold='nunique',
        
        # 고유 제품 목록을 ','로 구분한 문자쳘을 products column으로
        products= lambda x: ','.join(sorted(x.unique()))).reset_index()
    
    return result

# 2024-05-23

## 1517. Find Users With Valid E-Mails

In [ ]:
import pandas as pd

# 1. ^: 문자열의 시작
# 2. [A-Za-z]: 첫 번째 문자는 알파벳 대문자 또는 소문자
# 3. [A-Za-z0-9_\.\-]*: 첫 번째 문자 다음에는 알파벳, 숫자, 밑줄, 점, 또는 하이픈이 0개 이상 올 수 음
# 4. @: '@' 문자.
# 5. leetcode: '@' 다음에는 반드시 'leetcode'라는 문자열이 와야함
# 6. \.com: 'leetcode' 다음에는 반드시 '.com'이 와야함. 점은 이스케이프 처리되어 문자 그대로의 점을 의미
# 7. $: 문자열의 끝을 나타냄

def valid_emails(users: pd.DataFrame) -> pd.DataFrame:
    return users[users['mail'].str.match(r'^[A-Za-z][A-Za-z0-9_\.\-]*@leetcode\.com$')]

# 2024-06-19

## 1527. Patients With a Condition

In [ ]:
import pandas as pd

def find_patients(patients: pd.DataFrame) -> pd.DataFrame:
    # 'conditions' 열에 'DIAB1' 단어가 포함된 행을 필터링
    condition_filter = patients['conditions'].str.contains(r'\bDIAB1')
    
    # 필터 조건을 만족하는 행들로 구성된 df 반환
    return patients[condition_filter]

# 2024-06-21

## 1581. Customer Who Visited but Did Not Make Any Transactions

In [ ]:
import pandas as pd

def find_customers(visits: pd.DataFrame, transactions: pd.DataFrame) -> pd.DataFrame:
    # visit_id 기준 left join
    merged = pd.merge(visits, transactions, on='visit_id', how='left')
    
    # transaction_id가 NaN(결측값)인 행만 필터링
    filtered = merged.loc[merged['transaction_id'].isna()]
    
    # customer_id로 그룹화하여 각 고객의 방문 횟수(count_no_trans)를 계산하고 결과를 데이터프레임으로 반환
    result = filtered.groupby('customer_id').size().reset_index(name='count_no_trans')
    return result

In [ ]:
import pandas as pd

def find_customers(visits: pd.DataFrame, transactions: pd.DataFrame) -> pd.DataFrame:
    # visit_id 기준 left join
    merged = pd.merge(visits, transactions, on='visit_id', how='left')
    
    # transaction_id가 NaN(결측값)인 행만 필터링
    filtered = merged.loc[merged['transaction_id'].isna()]
    
    # customer_id로 그룹화하고, 각 customer_id에 대한 visit_id의 개수를 계산하여 새로운 열 count_no_trans를 생성
    result = filtered.groupby('customer_id', as_index=False).agg(count_no_trans=('visit_id','count'))
    return result

# 2024-07-30

## 1587. Bank Account Summary II

In [ ]:
import pandas as pd

def account_summary(users: pd.DataFrame, transactions: pd.DataFrame) -> pd.DataFrame:
    # account 열 기준으로 groupby 후 amount 열의 합을 balance 열로 집계
    summary = transactions.groupby('account', as_index=False).agg(balance=('amount', 'sum'))
    
    # account 열 기준으로 users와 summary 병합
    merged = pd.merge(users, summary, on='account', how='left')
    
    # balance가 10000 이상인 행만 필터링
    filtered = merged.loc[merged['balance']>10000]
    
    # 출력 요구 형식에 맞추어 account 열 제외
    result = filtered.drop('account', axis=1)
    return result

## 1633. Percentage of Users Attended a Contest

In [ ]:
import pandas as pd

def users_percentage(users: pd.DataFrame, register: pd.DataFrame) -> pd.DataFrame:
    # 전체 사용자 수 구하기
    total_num = users['user_id'].nunique()
    
    # 각 contest_id 별 사용자 수 계산
    summary = register.groupby('contest_id').size().reset_index(name='user_cnt')    
    
    # contest_id 별 percentage 계산
    summary['percentage'] = summary['user_cnt'] / total_num * 100
    
    # 비율 내림차순, contest_id 오름차순 정렬 후 출력 요구 형식에 맞추어 round
    result = summary[['contest_id', 'percentage']].sort_values(by=['percentage', 'contest_id'], ascending=[False, True]).round(2)
    return result

# 2024-07-31

## 1661. Average Time of Process per Machine

In [ ]:
import pandas as pd

def get_average_time(activity: pd.DataFrame) -> pd.DataFrame:
    # 'machine_id'를 기준으로 'activity' 데이터를 병합, 접미사로 구분
    merged = pd.merge(activity, activity, on='machine_id', suffixes=('_start', '_end'))
    
    # 'start'와 'end'가 위치한 행 필터링
    filtered = merged[(merged['activity_type_start'] == 'start') & (merged['activity_type_end'] == 'end')]
     
    # 'processing_time' 계산: 'timestamp_end'에서 'timestamp_start'를 뺌
    filtered['processing_time'] = filtered['timestamp_end'] - filtered['timestamp_start']
    
    # 'machine_id'별 평균 'processing_time'을 계산하고 소수점 세 자리로 반올림
    result = filtered.groupby('machine_id', as_index=False).agg(processing_time=('processing_time', 'mean')).round(3)
    return result

# 2024-08-01

## 1667. Fix Names in a Table

In [ ]:
import pandas as pd

def fix_names(users: pd.DataFrame) -> pd.DataFrame:
    # name 열 capitalize
    users['name'] = users['name'].str.capitalize()
    
    # user_id 기준 정렬
    result = users.sort_values('user_id')
    return result